 # P4 : Anticipez les besoins en consommation électrique de bâtiments

In [1]:
import os
import wget
import pandas as pd
from ast import literal_eval


In [2]:
write_data = True

# True : création d'un dossier Figures et Tableau
# dans lesquels seront créés les éléments qui serviront à la présentation
# et écriture des figures et tableaux dans ces dossier
#
# False : pas de création de dossier ni de figures ni de tableaux

if write_data is True:
    try:
        os.mkdir("./Figures/")
    except OSError as error:
        print(error)
    try:
        os.mkdir("./Tableaux/")
    except OSError as error:
        print(error)
else:
    print("""Visualisation uniquement dans le notebook
    pas de création de figures ni de tableaux""")


[Errno 17] File exists: './Figures/'
[Errno 17] File exists: './Tableaux/'


In [3]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe
# sinon récupération avec l'url
BEB2015 = pd.read_csv("2015-building-energy-benchmarking.csv")


In [4]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe
# sinon récupération avec l'url
BEB2016 = pd.read_csv("2016-building-energy-benchmarking.csv")


 Liste des colonnes présentes uniquement dans les données de 2015

In [5]:
# colonnes uniquement dans les données de 2015
BEB2015.columns.difference(BEB2016.columns)


Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'Location', 'OtherFuelUse(kBtu)', 'SPD Beats',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'Zip Codes'],
      dtype='object')

 Liste des colonnes présentes uniquement dans les données de 2016

In [6]:
# colonnes uniquement dans les données de 2016
BEB2016.columns.difference(BEB2015.columns)


Index(['Address', 'City', 'Comments', 'GHGEmissionsIntensity', 'Latitude',
       'Longitude', 'State', 'TotalGHGEmissions', 'ZipCode'],
      dtype='object')

 Les données de location en 2015 sont dans une seule colonne
 on va faire en sorte d'uniformiser avec les colonnes présentes en 2016

In [7]:
BEB2015["Location"] = [
    literal_eval(str(loc)) for index, loc in BEB2015.Location.iteritems()
]
BEB2015 = pd.concat(
    [
        BEB2015.drop(columns="Location", axis=1),
        BEB2015.Location.apply(pd.Series)
    ],
    axis=1,
)
BEB2015['human_address'] = [
    literal_eval(str(loc)) for index, loc in BEB2015.human_address.iteritems()
]
BEB2015 = pd.concat([
    BEB2015.drop(columns='human_address', axis=1),
    BEB2015.human_address.apply(pd.Series)
],
                    axis=1)


In [8]:
BEB2015 = BEB2015.rename(
    columns={
        "latitude": "Latitude",
        "longitude": "Longitude",
        "address": "Address",
        "city": "City",
        "state": "State",
        "zip": "ZipCode"
    })


In [9]:
BEB2015.columns.difference(BEB2016.columns)

Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'OtherFuelUse(kBtu)', 'SPD Beats',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'Zip Codes'],
      dtype='object')

In [10]:
BEB2016.columns.difference(BEB2015.columns)

Index(['Comments', 'GHGEmissionsIntensity', 'TotalGHGEmissions'], dtype='object')

 GHGEMissions (MetricTonsCO2e) et TotalGHGEmissions renseignent les mêmes informations
 ainsi que GHGEmissionsIntesity (kgCO2e/ft2) et GHGEmissionsIntensity

In [11]:
BEB2015.rename(
    columns={
        "GHGEmissions(MetricTonsCO2e)": "TotalGHGEmissions",
        "GHGEmissionsIntensity(kgCO2e/ft2)": "GHGEmissionsIntensity",
    },
    inplace=True,
)


In [12]:
BEB2015.columns.difference(BEB2016.columns)

Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'OtherFuelUse(kBtu)', 'SPD Beats',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'Zip Codes'],
      dtype='object')

In [13]:
BEB2016.columns.difference(BEB2015.columns)

Index(['Comments'], dtype='object')

In [14]:
BEB2016.Comments.unique()


array([nan])

 Pas de commentaire dans les données de 2016

In [15]:
BEB2016.drop(columns="Comments", inplace=True)

In [16]:
BEB2015.Comment.unique()

array([nan,
       'Under construction starting 6/2013 (old building demolished) and ending 9/2016. New Building re-opened 9/2016. Year built changed from 1948 to 2016.',
       "Part of McKinstry's campus, property operates a fabrication and production shop and is conditioned by natural gas overhead radiant unit heaters with no cooling. Learn more at www.mckinstry.com.",
       "One of two office buildings on McKinstry's campus partially remodeled in 2009 from warehouse to office space. Served by rooftop air handling units with underfloor VAVs.  Self-performed energy efficiency upgrades in 2015 include a long-term LED lighting re",
       'Part of McKinstry’s campus, this remodeled warehouse is mainly office but includes a full-size basketball court and gym with locker rooms, bistro with a full kitchen, and data center. Served by rooftop package units for heating and cooling. Upgrades inclu',
       'Construction completed in mid 2015. The building was unoccupied for most of 2015.',
 

 présence de commentaires dans les données de 2015

In [17]:
pd.DataFrame([BEB2015.dtypes, BEB2016.dtypes])


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,...,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,Latitude,Longitude,Address,City,State,ZipCode
0,int64,int64,object,object,object,object,int64,object,int64,int64,...,float64,float64,float64,int64,object,object,object,object,object,object
1,int64,int64,object,object,object,object,int64,object,int64,float64,...,NaN,NaN,NaN,NaN,float64,float64,object,object,object,float64


In [18]:
BEB2015[['Latitude', 'Longitude',
         'ZipCode']] = BEB2015[['Latitude', 'Longitude',
                                'ZipCode']].astype('float64')

 Nous allons joindre nos données pour n'avoir qu'un seul fichier
 sur lequel travailler lors des test de modèles

In [19]:
BEBFull = BEB2015.merge(BEB2016, how="outer")